<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analysis-of-holdings" data-toc-modified-id="Analysis-of-holdings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analysis of holdings</a></span><ul class="toc-item"><li><span><a href="#Other-functions" data-toc-modified-id="Other-functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Other functions</a></span></li><li><span><a href="#Other-functions" data-toc-modified-id="Other-functions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Other functions</a></span></li><li><span><a href="#Most-common-stocks" data-toc-modified-id="Most-common-stocks-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Most common stocks</a></span></li><li><span><a href="#Distribution-of-cusip-and-crsp_company_key" data-toc-modified-id="Distribution-of-cusip-and-crsp_company_key-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Distribution of cusip and crsp_company_key</a></span></li></ul></li></ul></div>

Findings:

Nearly all 'ord' stocks don't have perno or permco

Some stocks appear twice (at least based on name)

# Analysis of holdings

Description:



## Other functions

In [ ]:
import feather
import numpy as np
import pandas as pd
import pickle

from scipy import sparse

from sklearn.preprocessing import normalize, minmax_scale

import matplotlib.pyplot as plt

In [ ]:
path = '../data/processed/full.pickle'
pickle_off = open(path,"rb")
dict_all_years = pickle.load(pickle_off)
dict_year = dict_all_years[2018]

## Other functions

In [ ]:
year = 2010

row_info_f = dict_all_years[year]['row_info_f']
col_info_f = dict_all_years[year]['col_info_f']
returns_f = dict_all_years[year]['returns_f']
holdings_ft = dict_all_years[year]['holdings_ft']
holdings_b = dict_all_years[year]['holdings_b']
begin_date = dict_all_years[year]['begin_date']
end_date = dict_all_years[year]['end_date']

col_info_f = col_info_f.reset_index(drop=True)
col_info_f = col_info_f.assign(col = col_info_f.index)

row_info_f = row_info_f.reset_index(drop=True)
row_info_f = row_info_f.assign(row = row_info_f.index)                               

## Most common stocks

In [ ]:
print(row_info_f.shape)
print(col_info_f.shape)
print(holdings_b.shape)

In [ ]:
row_info_f.head()

In [ ]:
col_info_f.head()

In [ ]:
def most_common_stocks():
    # Change if you want to sum percentages instead of boolean values
    holdings_coo = holdings_b.tocoo()

    df_sparse = pd.DataFrame({'row' : holdings_coo.row,
                              'col' : holdings_coo.col,
                              'data' : holdings_coo.data})

    df_sparse = df_sparse.merge(row_info_f[['year','row','lipper_class']],how='left',on='row')

    no_unique_funds = row_info_f.shape[0]

    sum_col = (df_sparse
               .groupby(by = ['col'])
               .sum()
               .sort_values('data',ascending = False)
               .join(col_info_f[['security_name','col','cusip','permno','permco']],how='left')
               .assign(percent = lambda x:  x.data / no_unique_funds * 100)
               .drop(columns=['row','year','col'])
               .reset_index(drop=True)
               .head(100))

    print(
        'Most held stocks: ','\n\n'
        'Number of funds:    {}'.format(no_unique_funds),'\n'
    )

    return sum_col

In [ ]:
most_common_stocks().head()

In [ ]:
def most_common_stocks(holdings_b, row_info_f, col_info_f, year,lipper_class):
    "This prints a passed string into this function"
    # Enter date for which most common holdings are calculated
    year = year
    lipper_class = lipper_class

    # Change if you want tu sum percentages instead of boolean values
    holdings_coo = holdings_b.tocoo()

    df_sparse = pd.DataFrame({'row' : holdings_coo.row,
                              'col' : holdings_coo.col,
                              'data' : holdings_coo.data})

    df_sparse = df_sparse.merge(row_info_f[['year','row','lipper_class']],how='left',on='row')

    my_filter = '''year == @year and lipper_class == @lipper_class '''
    no_unique_funds = row_info_f.query(my_filter).shape[0]

    sum_col = (df_sparse
               .query(my_filter)
               .groupby(by = ['col'])
               .sum()
               .sort_values('data',ascending = False)
               .join(col_info_f[['security_name','col']],how='left')
               .assign(percent = lambda x:  x.data / no_unique_funds * 100)
               .drop(columns=['row','year','col'])
               .reset_index(drop=True)
               .head(3))
    
    sum_col['Lipper class'] = lipper_class
    sum_col = sum_col.iloc[:,[3,1,0,2]]
    sum_col.columns = ['Lipper class','Security name','Nr. of funds with holding','Percentage of funds']

    # print(
    #     'Most held stocks: ','\n\n'
    #     'Lipper class:       {}'.format(lipper_class),'\n'
    #     'Number of funds:    {}'.format(no_unique_funds),'\n'
    #     'Year:               {}'.format(year))

    return sum_col

In [ ]:
options = {
    'selected_obj_codes'         : ('EIEI',
                                    'LCVE', 'LCCE', 'LCGE',
                                    'MLVE', 'MLCE', 'MLGE',
                                    'MCVE', 'MCCE', 'MCGE',
                                    'SCVE', 'SCCE', 'SCGE')}
results = []


for i, code in enumerate(options.get('selected_obj_codes')):
    year = 2014

    row_info_f = dict_all_years[year]['row_info_f']
    col_info_f = dict_all_years[year]['col_info_f']
    holdings_b = dict_all_years[year]['holdings_b']

    col_info_f = col_info_f.reset_index(drop=True)
    col_info_f = col_info_f.assign(col = col_info_f.index)

    row_info_f = row_info_f.reset_index(drop=True)
    row_info_f = row_info_f.assign(row = row_info_f.index)  
    
    
    results.append(most_common_stocks(holdings_b, row_info_f, col_info_f, year=year, lipper_class=code))

In [ ]:
results_df = pd.concat(results)
results_df_14 = results_df.iloc[:,[0,1,3]]

In [ ]:
results_df_comb = pd.concat([results_df_10,results_df_14.iloc[:,[1,2]],results_df_18.iloc[:,[1,2]]],1)
results_df_comb

In [ ]:
print(round(results_df_comb,2).to_latex(index = False,
                    index_names = False,
                    bold_rows = False))

In [ ]:
holdings_summary = pd.DataFrame(holdings_ft.sum(0).T)
holdings_summary.columns = ['total']
holdings_summary.shape

## Distribution of cusip and crsp_company_key

In [ ]:
col_info.sample()

In [ ]:
print('Percentage of identifiers missing:')
np.sum(col_info[['crsp_company_key','cusip','permno','permco']].isna()) / col_info.shape[0] * 100

In [ ]:
print('Percentage of stocks with same cusip appearing twice:')
col_info.drop_duplicates(subset='cusip').shape[0] / col_info.shape[0] * 100

In [ ]:
print('Percentage of stocks with same SHORT cusip appearing twice:')
col_info['short_cusip'] = col_info['cusip'].astype(str).str[:6]
col_info.drop_duplicates(subset='short_cusip').shape[0] / col_info.shape[0] * 100

In [ ]:
col_info

In [ ]:
t = 12_000
col_info.sort_values('security_name').iloc[t:t+100,:]

In [ ]:
col_info[col_info['security_name'].str.contains('ALLERGAN')]

In [ ]:
col_info['security_name']

In [ ]:
col_info['security_name'].drop_duplicates()

In [ ]:
col_info_f.shape

In [ ]:
print(np.sum(col_info_f['permco'].isna()))
col_info_f[col_info_f['permco'].isna()]

In [ ]:
np.sum(col_info_f['crsp_company_key'].isna())

In [ ]:
col_info_f['permco'] = col_info_f['permco'].astype(float)
col_info_f['permno'] = col_info_f['permno'].astype(float)
ccm_link['gvkey'] = ccm_link['gvkey'].astype(float)

In [ ]:
ccm_link.head()

In [ ]:
ccm_link['gvkey'] = ccm_link['gvkey'].astype(float)

In [ ]:
ccm_link_f = ccm_link.dropna(subset = ['lpermno'])

In [ ]:
col_info_fm = col_info_f.merge(ccm_link_f[['gvkey','lpermno']], how = 'left', left_on = ['permno'], right_on = ['lpermno'])

In [ ]:
ccm_link_f[ccm_link_f['gvkey'] == 5047.0]

In [ ]:
col_info_fm

In [ ]:
col_info_fm.drop_duplicates().shape